In [13]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-3-5-haiku-latest"
num_tokens = 1000

In [14]:
# Helper functions
def add_user_message(messages, text):
    user_message = {"role": "user", "content": text}
    messages.append(user_message)


def add_assistant_message(messages, text):
    assistant_message = {"role": "assistant", "content": text}
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": num_tokens,
        "messages": messages,
        "temperature": temperature,
    }

    if system:
        params["system"] = system
    
    if stop_sequences:
        params["stop_sequences"] = stop_sequences

    with client.messages.stream(**params) as stream:
        for text in stream.text_stream:
            pass
    response = stream.get_final_message()
    #message = stream.get_final_message()
    
    # message = client.messages.create(**params)
    return response.content[0].text
    # return message.content[0].text

In [ ]:
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing a task that requires Python, JSON, or a Regex to complete.  Include a set of solution
criteria describing the necessary traits required for a concise, clear, and comprehensive solution.

Example output:
```json
[
    {
        "task": "Description of task",
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""
    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [16]:
dataset = generate_dataset()
print(dataset)

[{'task': 'Create a Python function to convert an AWS EC2 instance state code (e.g., 0, 16, 32, 64, 80) to its human-readable description (pending, running, shutting-down, terminated, stopping)'}, {'task': 'Write a JSON object that defines an AWS IAM policy allowing read-only access to S3 buckets in a specific AWS account'}, {'task': 'Develop a regular expression that validates AWS resource naming conventions, specifically for validating EC2 instance names (lowercase letters, numbers, hyphens, 1-63 characters long)'}]


In [17]:
with open('dataset.json', 'w') as f:
   json.dump(dataset, f, indent=2)